<a href="https://colab.research.google.com/github/rajkiran15/colab-hands-on/blob/main/animal_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Important rules:
    # The folder name = class label (cats, dogs, 0, 1, etc.)
    # Inside each class folder → put images of only that class
    # Any image format is fine: .jpg, .jpeg, .png

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
from PIL import Image

# ------------ SETTINGS ------------ #
# This folder should contain subfolders like: cats/, dogs/
DATA_DIR = "my_images"   # change this if your folder has different name
IMG_SIZE = (128, 128)    # all images will be resized to 128x128
BATCH_SIZE = 32          # how many images we load at once
MODEL_PATH = "my_trained_model.keras"
TEST_DIR = "test_images"

# ------------ LOAD TRAIN & VALIDATION DATASETS ------------ #
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,     # 20% of images will be used as validation
    subset="training",
    seed=123,                 # random shuffle seed (to keep it reproducible)
    image_size=IMG_SIZE,      # resize all images to same size
    batch_size=BATCH_SIZE     # load this many images at once
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# ------------ CHECK WHAT GOT LOADED ------------ #
class_names = train_ds.class_names
print("✅ Found classes:", class_names)

for images, labels in train_ds.take(1):
    print("🔹 One batch image tensor shape:", images.shape)
    print("🔹 One batch label tensor shape:", labels.shape)
    break



# --------------- Build CNN Model --------------- #
model = keras.Sequential([
    layers.Rescaling(1./255, input_shape=(128, 128, 3)),  # normalize if not normalized yet
    layers.Conv2D(32, activation='relu', kernel_size=3),
    layers.MaxPooling2D(),

    layers.Conv2D(64, activation='relu', kernel_size=3),
    layers.MaxPooling2D(),

    layers.Conv2D(128, activation='relu', kernel_size=3),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(train_ds.class_names), activation='softmax')  # output = number of classes
])

model.summary()


# --------------- Compile Model --------------- #
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("\n🚀 Training on your images...")
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

# --------------- Save the trained model --------------- #
model.save(MODEL_PATH)
print("\n✅ Model saved as: my_trained_model.keras")

# --------------- LOAD MODEL AGAIN --------------- #
print("\n🔁 Loading saved model for inference...")
model = keras.models.load_model(MODEL_PATH)
print("✅ Model loaded!")

# Re-attach class names to model so we can print labels during prediction
model.class_names = class_names

# --------------- PREDICTION FUNCTION --------------- #
def predict_image(img_path, size=(128,128)):
    img = Image.open(img_path).convert("RGB")
    img = img.resize(size)
    img_array = np.array(img).reshape(1, 128, 128, 3)
    pred = model.predict(img_array)
    return model.class_names[pred.argmax()]

# --------------- PREDICT ON TEST FOLDER --------------- #
if os.path.exists(TEST_DIR):
    print("\n🔍 Predicting images from:", TEST_DIR)
    for file in os.listdir(TEST_DIR):
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            path = os.path.join(TEST_DIR, file)
            result = predict_image(path)
            print(f"🖼 {file} → Prediction:", result)
else:
    print("\n⚠ No test folder found:", TEST_DIR)




Found 14 files belonging to 2 classes.
Using 12 files for training.
Found 14 files belonging to 2 classes.
Using 2 files for validation.
✅ Found classes: ['cats', 'dogs']
🔹 One batch image tensor shape: (12, 128, 128, 3)
🔹 One batch label tensor shape: (12,)


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_5 (Rescaling)         │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,898 (12.61 MB)

 Trainable params: 3,304,898 (12.61 MB)

 Non-trainable params: 0 (0.00 B)


🚀 Training on your images...
Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.6667 - loss: 0.6641 - val_accuracy: 0.5000 - val_loss: 1.5043
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 680ms/step - accuracy: 0.5833 - loss: 1.2433 - val_accuracy: 0.5000 - val_loss: 0.9391
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 649ms/step - accuracy: 0.4167 - loss: 0.7886 - val_accuracy: 0.5000 - val_loss: 0.7271
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 478ms/step - accuracy: 0.6667 - loss: 0.6095 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 437ms/step - accuracy: 0.6667 - loss: 0.5739 - val_accuracy: 0.5000 - val_loss: 0.7084

✅ Model saved as: my_trained_model.keras

🔁 Loading saved model for inference...
✅ Model loaded!

🔍 Predicting images from: test_images
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
🖼 two_1.jpg → Prediction: dogs
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
🖼 one_1.jpg → Prediction: dogs
